## Лабораторная работа №3. Feature engineering. Part 2.

### Задание.
***
**Цель лабораторной работы:** изучение продвинутых способов предварительной обработки данных для дальнейшего формирования моделей.

**Задание:**  
Для выбранного датасета (датасетов) на основе материалов лекций решить следующие задачи:
1. масштабирование признаков (не менее чем тремя способами);  
2. обработку выбросов для числовых признаков (по одному способу для удаления выбросов и для замены выбросов);  
3. обработку по крайней мере одного нестандартного признака (который не является числовым или категориальным);  
4. отбор признаков:  
  * один метод из группы методов фильтрации (filter methods);  
  * один метод из группы методов обертывания (wrapper methods);  
  * один метод из группы методов вложений (embedded methods).

### Выполнение.
***
Задание буду выполнять на датасете ["Computer Parts (CPUs and GPUs)"](https://www.kaggle.com/iliassekkaf/computerparts). Датасет содержит значения различных характеристик по CPU GPU. C данным датасетом я работал в ЛР2, а в данной ЛР я продолжу обрабатывать данный датасет, используя сохраненные наработки из ЛР2.

Загрузка и обзор датасета, сохраненного в конце ЛР2:

In [1]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import scipy.stats as stats
%matplotlib inline 
sns.set(style="ticks")

dataset = pd.read_csv('../data/cpu-dataset-from-lr2.csv', sep=",")
dataset

,Product_Collection,Vertical_Segment,Status,Launch_Date,Lithography,nb_of_Cores,Processor_Base_Frequency,Cache,TDP,Embedded_Options_Available,T,Intel_Hyper_Threading_Technology_,Intel_Virtualization_Technology_VTx_,Intel_64_,Instruction_Set,T_bc
0,13,Mobile,927,Q3'16,14 nm,2,1.30 GHz,4 MB SmartCache,4.5 W,No,100.0,Yes,Yes,Yes,64-bit,4.605170
1,15,Mobile,927,Q3'17,14 nm,4,1.60 GHz,6 MB SmartCache,15 W,No,100.0,Yes,Yes,Yes,64-bit,4.605170
2,16,Mobile,927,Q3'17,14 nm,4,1.80 GHz,8 MB SmartCache,15 W,No,100.0,Yes,Yes,Yes,64-bit,4.605170
3,29,Desktop,415,Q1'12,32 nm,4,3.60 GHz,10 MB SmartCache,130 W,No,66.8,Yes,Yes,Yes,64-bit,4.201703
4,12,Mobile,927,Q1'17,14 nm,2,1.20 GHz,4 MB SmartCache,4.5 W,No,100.0,Yes,Yes,Yes,64-bit,4.605170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053,10,Mobile,927,Q3'15,14 nm,2,1.10 GHz,4 MB SmartCache,4.5 W,No,100.0,Yes,Yes,Yes,64-bit,4.605170
2054,10,Mobile,927,Q3'15,14 nm,2,1.10 GHz,4 MB SmartCache,4.5 W,No,100.0,Yes,Yes,Yes,64-bit,4.605170
2055,10,Mobile,927,Q3'15,14 nm,2,1.20 GHz,4 MB SmartCache,4.5 W,No,100.0,Yes,Yes,Yes,64-bit,4.605170
2056,6,Mobile,927,Q1'15,14 nm,2,2.00 GHz,4 MB SmartCache,15 W,No,105.0,Yes,Yes,Yes,64-bit,4.653960


### Обработка одного нестандартного признака.  
***
Таким признаком (помимо Т, обработанного в ЛР2) является признак *Processor_Base_Frequency*. В датасете это не числовое значение (type `object`), но на деле представляет собой `float` значение. Причем частоты представлены в MHz и GHz, что также требует конвертации.  
Попробуем распрарсить числовые значения и привести их к одинаковой размерности - MHz.